# Table of Contents
 <p><div class="lev1 toc-item"><a href="#The-plan" data-toc-modified-id="The-plan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The plan</a></div><div class="lev2 toc-item"><a href="#Results" data-toc-modified-id="Results-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Results</a></div><div class="lev2 toc-item"><a href="#Baseline-model" data-toc-modified-id="Baseline-model-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Baseline model</a></div><div class="lev2 toc-item"><a href="#Image-Augmentation" data-toc-modified-id="Image-Augmentation-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Image Augmentation</a></div><div class="lev2 toc-item"><a href="#Augmenting-feature-maps" data-toc-modified-id="Augmenting-feature-maps-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Augmenting feature maps</a></div><div class="lev2 toc-item"><a href="#Submission" data-toc-modified-id="Submission-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Submission</a></div><div class="lev3 toc-item"><a href="#Baseline-model" data-toc-modified-id="Baseline-model-151"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Baseline model</a></div><div class="lev3 toc-item"><a href="#Augmented-feature-maps" data-toc-modified-id="Augmented-feature-maps-152"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Augmented feature maps</a></div>

# The plan

In this notebook we will look at how data augmentation in feature maps space can improve performance of a classifier. 
See the blog post on justification when we need such transformations of feature maps.

To see whether we have any gains I'm going to do the following:

Use VGG with fixed weights as pre-trained network and will develop three model on top of it

1. The simple fully convolutional baseline model
2. The model above trained on additional images produced by applying image augmenation of the original images
3. The same model but trained with transformations of feature maps


The models will be trained on small subsets(10,20,40,80 samples) of Kaggle's Cat&Dogs dataset.

Then I will compare the performance of the models and their training time. 



## Results

In [220]:
#train_samples, model, accuracy, sec per epoch
results = [(10, 'bs', 0.67,0),
    (10,'img_aug', 0.86,11),
    (10,'fm_aug', 0.81,0),
    #(10,'fm_aug_flex',0.74,0),

    (20,'bs',0.78,0),
    (20,'img_aug', 0.92,11),
    (20,'fm_aug',0.92,0),

    (40,'bs', 0.92, 0),
    (40,'img_aug',0.94, 12),
    (40,'fm_aug',0.93,0),


    (80,'bs', 0.94, 0),
    (80,'img_aug',0.94, 15),
    (80,'fm_aug', 0.94, 0)]

In [223]:
df = pd.DataFrame(results, columns=['n_samples','model','accuracy','time'])
df.shape
df.head()

(12, 4)

,n_samples,model,accuracy,time
0,10,bs,0.67,0
1,10,img_aug,0.86,11
2,10,fm_aug,0.81,0
3,20,bs,0.78,0
4,20,img_aug,0.92,11


In [228]:
#print(df.to_html())

In [229]:
df = df.replace({'model':{'bs':'baseline','img_aug':'image augmentation','fm_aug':'feature maps augmentation'}})

In [232]:
df
print(df.to_html(index = False))

,n_samples,model,accuracy,time
0,10,baseline,0.67,0
1,10,image augmentation,0.86,11
2,10,feature maps augmentation,0.81,0
3,20,baseline,0.78,0
4,20,image augmentation,0.92,11
5,20,feature maps augmentation,0.92,0
6,40,baseline,0.92,0
7,40,image augmentation,0.94,12
8,40,feature maps augmentation,0.93,0
9,80,baseline,0.94,0


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>n_samples</th>
      <th>model</th>
      <th>accuracy</th>
      <th>time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10</td>
      <td>baseline</td>
      <td>0.67</td>
      <td>0</td>
    </tr>
    <tr>
      <td>10</td>
      <td>image augmentation</td>
      <td>0.86</td>
      <td>11</td>
    </tr>
    <tr>
      <td>10</td>
      <td>feature maps augmentation</td>
      <td>0.81</td>
      <td>0</td>
    </tr>
    <tr>
      <td>20</td>
      <td>baseline</td>
      <td>0.78</td>
      <td>0</td>
    </tr>
    <tr>
      <td>20</td>
      <td>image augmentation</td>
      <td>0.92</td>
      <td>11</td>
    </tr>
    <tr>
      <td>20</td>
      <td>feature maps augmentation</td>
      <td>0.92</td>
      <td>0</td>
    </tr>
    <tr>
      <td>40</td>
      <td>baseline</td>
      <td>0.92</td>
      <td>0</td>
    </tr>
    <tr>
      <td>40</td>
      <td>image augmentatio

In [234]:
0.16 / 0.36

0.4444444444444445


Francois Chollet in [his tutorial]() used Kaggle [Dogs vs Cats Redux](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data) competition data to show how to use image augmentation and pre-trained models to train a classifier small amount of data. Francois used only 2,000 (1000 cats and 1000 dogs) out of 25,000 images to achieve ... accuracy 

I will make the task even more challenging I will train a classifier on small number of examples starting from 5 samples per class, 10, 20, and 40 samples per class

In [49]:
import sys
sys.path.append('..')
from cat_dog.util_blog import *

import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fpath = full_path('train/*.jpg')

In [3]:
#fpath = full_path('train/*.jpg')
fpath = '../../data/cat_dog/train/*.jpg'
img_df = pd.DataFrame({'path':glob.glob(fpath)})
img_df['dog'] = (img_df.path.str.find('train/dog.') > 0).astype(int)
img_df.shape

(25000, 2)

In [4]:
fpath = full_path('test/*.jpg')
test_img_df = pd.DataFrame({'path':glob.glob(fpath)})
test_img_df.shape

(12500, 1)

In [177]:
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
test_idx, tmp_idx = next(StratifiedShuffleSplit(train_size = 0.1, test_size = 0.1, random_state= 517).split(img_df, img_df.dog))


In [179]:
tmp_idx.shape

(2500,)

In [180]:
train_idx, val_idx = train_test_split(tmp_idx, train_size = 80, test_size=500, random_state=801, 
                                     stratify = img_df.iloc[tmp_idx].dog)
#next(ShuffleSplit(test_size = 0.8, random_state= 617).split(val_idx, val_idx))
val_idx.shape
test_idx.shape
train_idx.shape

(500,)

(2500,)

(80,)

In [181]:
for idx in [test_idx, val_idx,train_idx]:
    img_df.loc[idx].shape
    np.mean(img_df.loc[idx].dog)
    

(2500, 2)

0.5

(500, 2)

0.5

(80, 2)

0.5

In [182]:
sum(np.intersect1d(train_idx,test_idx))
sum(np.intersect1d(train_idx,val_idx))
sum(np.intersect1d(test_idx,val_idx))

0

0

0

In [183]:
val_df = img_df.iloc[val_idx]
val_df.shape
test_df = img_df.iloc[test_idx]
test_df.shape
train_df = img_df.iloc[train_idx]
train_df.shape

(500, 2)

(2500, 2)

(80, 2)

In [15]:
img_dims = [load_img(p).size for p in train_df.path]
from collections import Counter
c = Counter(img_dims)
from operator import itemgetter
c_sort = sorted(c.items(), key=itemgetter(1), reverse = True)
c_sort[:10]

[((500, 374), 5),
 ((499, 375), 4),
 ((319, 240), 2),
 ((299, 240), 1),
 ((500, 365), 1),
 ((500, 332), 1),
 ((347, 500), 1),
 ((178, 184), 1),
 ((320, 440), 1),
 ((339, 260), 1)]

In [435]:
374/16

23.375

In [133]:
#target_size = (500, 374)
target_size = (374,500)

In [184]:
X_train, y_train = prep_x_y(train_df, target_size = target_size)
X_train.shape, y_train.shape

X_val, y_val = prep_x_y(val_df, target_size = target_size)
X_val.shape, y_val.shape


((80, 374, 500, 3), (80,))

((500, 374, 500, 3), (500,))

In [135]:

X_test, y_test = prep_x_y(test_df, target_size = target_size)
X_test.shape, y_test.shape

((500, 374, 500, 3), (500,))

((2500, 374, 500, 3), (2500,))

## Baseline model

The first model is to set a baseline it will be a full convolution network attached to top of pre-trained VGG
As I'm not going to use image augmenation for the baseline I will first save run VGG for train and test sets and will keep output tensors as input to the model.


In [136]:
#conv_out = attach_top_to_vgg(lambda x: x, layer_name = 'block5_pool')
#conv_out = attach_top_to_vgg(lambda x: x, layer_name = 'block5_pool')
conv_out = attach_top_to_vgg(lambda x: x, layer_name = 'block5_conv3')

In [185]:
X2_val = conv_out.predict(X_val, batch_size=24)
X2_val.shape

X2_train = conv_out.predict(X_train, batch_size=24)
X2_train.shape

(500, 23, 31, 512)

(80, 23, 31, 512)

In [186]:
%time X2_test = conv_out.predict(X_test, batch_size=24)
X2_test.shape

CPU times: user 53.6 s, sys: 12.1 s, total: 1min 5s
Wall time: 1min 1s


(2500, 23, 31, 512)

In [187]:
def fc_bin_m1(x):
    """
    Fully Convolutional binary model
    """
    x = MaxPooling2D((2, 2), strides=(2, 2), name='b1_pool')(x)
    x = Convolution2D(256, 1, 1, activation='relu', name='conv_1x1')(x)
    x = BatchNormalization(axis=3)(x)
    x = Convolution2D(128, 7, 7, activation='relu', border_mode='valid', name='conv_FC1')(x)
    x = BatchNormalization(axis=3)(x)
    x = Convolution2D(1, 1, 1, activation='sigmoid', border_mode='valid', name='conv_FC_last')(x)
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    return x

In [188]:
input_shape = (None, None, 512)
#model = fc_bin_m2(Input(shape=input_shape), crop_size)
m_input = Input(shape=input_shape)
model = Model(m_input, fc_bin_m1(m_input))

In [189]:
model.compile(loss='binary_crossentropy',
              optimizer= Adam(lr=0.001),
              metrics=['accuracy'])

In [190]:
model.optimizer.lr = 1e-4
history = model.fit(X2_train, y_train, validation_data=(X2_val, y_val),
                    batch_size = 32,
                    nb_epoch = 20, verbose = 2)


Train on 80 samples, validate on 500 samples
Epoch 1/20
1s - loss: 0.5641 - acc: 0.7500 - val_loss: 1.1526 - val_acc: 0.5760
Epoch 2/20
0s - loss: 0.2637 - acc: 0.9375 - val_loss: 1.0563 - val_acc: 0.6660
Epoch 3/20
0s - loss: 0.1984 - acc: 0.9500 - val_loss: 0.5350 - val_acc: 0.7940
Epoch 4/20
0s - loss: 0.0950 - acc: 1.0000 - val_loss: 0.3682 - val_acc: 0.8480
Epoch 5/20
0s - loss: 0.1204 - acc: 0.9500 - val_loss: 0.3003 - val_acc: 0.8740
Epoch 6/20
0s - loss: 0.0557 - acc: 1.0000 - val_loss: 0.2795 - val_acc: 0.8840
Epoch 7/20
0s - loss: 0.0430 - acc: 1.0000 - val_loss: 0.2638 - val_acc: 0.8900
Epoch 8/20
0s - loss: 0.0354 - acc: 1.0000 - val_loss: 0.2450 - val_acc: 0.8900
Epoch 9/20
0s - loss: 0.0435 - acc: 0.9875 - val_loss: 0.2137 - val_acc: 0.9040
Epoch 10/20
0s - loss: 0.0319 - acc: 1.0000 - val_loss: 0.1816 - val_acc: 0.9100
Epoch 11/20
0s - loss: 0.0191 - acc: 1.0000 - val_loss: 0.1670 - val_acc: 0.9220
Epoch 12/20
0s - loss: 0.0165 - acc: 1.0000 - val_loss: 0.1642 - val_acc:

In [191]:
model.optimizer.lr = 1e-5
history = model.fit(X2_train, y_train, validation_data=(X2_val, y_val),
                    nb_epoch = 10, verbose = 2)

Train on 80 samples, validate on 500 samples
Epoch 1/10
0s - loss: 0.0047 - acc: 1.0000 - val_loss: 0.1592 - val_acc: 0.9440
Epoch 2/10
0s - loss: 0.0250 - acc: 1.0000 - val_loss: 0.1579 - val_acc: 0.9420
Epoch 3/10
0s - loss: 0.0178 - acc: 1.0000 - val_loss: 0.1599 - val_acc: 0.9460
Epoch 4/10
0s - loss: 0.0065 - acc: 1.0000 - val_loss: 0.1623 - val_acc: 0.9440
Epoch 5/10
0s - loss: 0.0031 - acc: 1.0000 - val_loss: 0.1646 - val_acc: 0.9400
Epoch 6/10
0s - loss: 0.0034 - acc: 1.0000 - val_loss: 0.1660 - val_acc: 0.9320
Epoch 7/10
0s - loss: 0.0038 - acc: 1.0000 - val_loss: 0.1673 - val_acc: 0.9320
Epoch 8/10
0s - loss: 0.0035 - acc: 1.0000 - val_loss: 0.1673 - val_acc: 0.9360
Epoch 9/10
0s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.1669 - val_acc: 0.9440
Epoch 10/10
0s - loss: 0.0022 - acc: 1.0000 - val_loss: 0.1660 - val_acc: 0.9460


In [193]:
model.optimizer.lr = 1e-6
history = model.fit(X2_train, y_train, validation_data=(X2_val, y_val),
                    nb_epoch = 5, verbose = 2)

Train on 80 samples, validate on 500 samples
Epoch 1/5
0s - loss: 0.0021 - acc: 1.0000 - val_loss: 0.1650 - val_acc: 0.9440
Epoch 2/5
0s - loss: 0.0048 - acc: 1.0000 - val_loss: 0.1639 - val_acc: 0.9420
Epoch 3/5
0s - loss: 0.0028 - acc: 1.0000 - val_loss: 0.1627 - val_acc: 0.9440
Epoch 4/5
0s - loss: 0.0018 - acc: 1.0000 - val_loss: 0.1613 - val_acc: 0.9440
Epoch 5/5
0s - loss: 0.0048 - acc: 1.0000 - val_loss: 0.1606 - val_acc: 0.9440


In [194]:
pred = model.predict(X2_test)
pred.shape
pred = reduce_conf(pred)
metrics.log_loss(y_test, pred)
pred_class = (pred > 0.5).astype('float32')
metrics.accuracy_score(y_test, pred_class)

(2500, 1)

0.19046315607689321

0.93600000000000005

In [204]:
from sklearn.metrics import classification_report
print(classification_report(y_test, (pred > 0.5).astype('int')))

             precision    recall  f1-score   support

        0.0       0.96      0.93      0.94      1250
        1.0       0.93      0.96      0.94      1250

avg / total       0.94      0.94      0.94      2500



We have got a very strong baseline with very simple and fast model. Thank to using pre-trained model.
But let's see if we can improve that with image augmentation. 

## Image Augmentation

We are going to use Keras ImageDataGenerator to do the following augmenations:

Why those?
Because they are common on ImageNet Challenge. And also because similar transformations could be applied to feature maps. So we are giving feature map transformations a fair chance.  


In [195]:
train_datagen = ImageDataGenerator(
        #shear_range=0.2,
        width_shift_range=0.2,
        height_shift_range =0.2,
        horizontal_flip=True)

Because of image augmentation we cannot use prepared output of VGG convolution layers. So we need to attach our model and it is the same model but the training starts from the image level. And for every batch we run the input through the all layer of VGG - it takes time. 

In [196]:

model = attach_top_to_vgg(fc_bin_m1, layer_name = 'block5_conv3')
#model = attach_top_to_vgg(fc_bin_m1, layer_name = 'block5_pool')

In [197]:
model.compile(loss='binary_crossentropy',
              optimizer= Adam(lr=0.001),
              metrics=['accuracy'])

In [198]:
model.optimizer.lr = 1e-4
history = model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=16),
                    samples_per_epoch=len(X_train), nb_epoch = 9, verbose = 2,
                    validation_data=(X_val, y_val))

Epoch 1/9
15s - loss: 0.5107 - acc: 0.7500 - val_loss: 1.7492 - val_acc: 0.5300
Epoch 2/9
14s - loss: 0.2671 - acc: 0.9375 - val_loss: 0.7316 - val_acc: 0.7460
Epoch 3/9
14s - loss: 0.1976 - acc: 0.9625 - val_loss: 0.4006 - val_acc: 0.8380
Epoch 4/9
14s - loss: 0.1534 - acc: 0.9875 - val_loss: 0.2385 - val_acc: 0.8880
Epoch 5/9
14s - loss: 0.1260 - acc: 1.0000 - val_loss: 0.2120 - val_acc: 0.8980
Epoch 6/9
14s - loss: 0.1063 - acc: 1.0000 - val_loss: 0.2024 - val_acc: 0.9080
Epoch 7/9
14s - loss: 0.0768 - acc: 1.0000 - val_loss: 0.1861 - val_acc: 0.9180
Epoch 8/9
14s - loss: 0.0759 - acc: 0.9875 - val_loss: 0.1771 - val_acc: 0.9220
Epoch 9/9
14s - loss: 0.0589 - acc: 1.0000 - val_loss: 0.1787 - val_acc: 0.9240


In [199]:
model.optimizer.lr = 1e-5
history = model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=16),
                    samples_per_epoch=len(X_train), nb_epoch = 4, verbose = 2,
                    validation_data=(X_val, y_val))

Epoch 1/4
14s - loss: 0.0659 - acc: 1.0000 - val_loss: 0.1821 - val_acc: 0.9240
Epoch 2/4
14s - loss: 0.0765 - acc: 1.0000 - val_loss: 0.1803 - val_acc: 0.9220
Epoch 3/4
14s - loss: 0.0520 - acc: 1.0000 - val_loss: 0.1773 - val_acc: 0.9240
Epoch 4/4
14s - loss: 0.0560 - acc: 1.0000 - val_loss: 0.1725 - val_acc: 0.9280


In [202]:
model.optimizer.lr = 1e-6
history = model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=16),
                    samples_per_epoch=len(X_train), nb_epoch = 5, verbose = 2,
                    validation_data=(X_val, y_val))

Epoch 1/5
14s - loss: 0.0261 - acc: 1.0000 - val_loss: 0.1549 - val_acc: 0.9500
Epoch 2/5
14s - loss: 0.0256 - acc: 1.0000 - val_loss: 0.1538 - val_acc: 0.9520
Epoch 3/5
14s - loss: 0.0309 - acc: 1.0000 - val_loss: 0.1485 - val_acc: 0.9560
Epoch 4/5
14s - loss: 0.0292 - acc: 1.0000 - val_loss: 0.1416 - val_acc: 0.9580
Epoch 5/5
14s - loss: 0.0279 - acc: 1.0000 - val_loss: 0.1405 - val_acc: 0.9600


In [203]:
pred = model.predict(X_test,batch_size=24)
pred.shape
pred = reduce_conf(pred)
metrics.log_loss(y_test, pred)
pred_class = (pred > 0.5).astype('float32')
metrics.accuracy_score(y_test, pred_class)


(2500, 1)

0.16537814592011274

0.94279999999999997

You have to be patient - training gets 100 times slower. 

## Augmenting feature maps

Now we are going to apply similar spatial transformation to feature maps extracted by the pre-trained model

In [30]:
def aug_fun(x, cr = (0.1,0.1)):#crop_shape=(12,9)):
    shape = (x.shape[1],x.shape[2]) #expects a batch of image tensors
    crop_shape = ((int(shape[0] - cr[0]*shape[0])), (int(shape[1] - cr[1]*shape[1])))
    #print(crop_shape)
    stride = [0,0]
    stride[0] = np.random.randint(shape[0] - crop_shape[0] + 1)
    stride[1] = np.random.randint(shape[1] - crop_shape[1] + 1)
    #print("sride:", stride)
    res = x[:,stride[0]:(crop_shape[0]+stride[0]),stride[1]:(crop_shape[1]+stride[1]),:]
    #and random flip
    if np.random.binomial(1,0.5):
        #res = np.fliplr(res)
        #res = res[:,::-1]
        res = np.flip(res, axis = 2)
    return res
        

In [105]:
def conv_crop_gen(X, y, cr = (0.2,0.2)):
    while True:
        for i in np.random.permutation(X.shape[0]):
            res = (aug_fun(X[i:i+1,::], cr), y[i:i+1])
            yield res

In [47]:
def aug_fun_flex(x, cr = (0.1,0.1)):#crop_shape=(12,9)):
    shape = (x.shape[1],x.shape[2]) #expects a batch of image tensors
    crop_shape = (shape[0] - np.random.randint(math.ceil(cr[0]*shape[0])+1), 
                  shape[1] - np.random.randint(math.ceil(cr[1]*shape[1])+1))
    #print(crop_shape)
    stride = [0,0]
    stride[0] = np.random.randint(shape[0] - crop_shape[0] + 1)
    stride[1] = np.random.randint(shape[1] - crop_shape[1] + 1)
    #print("sride:", stride)
    res = x[:,stride[0]:(crop_shape[0]+stride[0]),stride[1]:(crop_shape[1]+stride[1]),:]
    #and random flip
    if np.random.binomial(1,0.5):
        #res = np.fliplr(res)
        res = np.flip(res, axis = 2)
        #res = res[:,::-1]
    return res

In [54]:
def crop_gen_flex(X, y, cr = (0.3,0.3), batch_size = 8):
    while True:
        idx = np.random.permutation(X.shape[0])
        for j in range(0, X.shape[0], batch_size):
            #print(j)
            b_idx = idx[j:(j+batch_size)]
            res = (aug_fun_flex(X[b_idx,::], cr), y[b_idx])
            yield res

In [205]:
train_gen = gen_batch_wrap(conv_crop_gen(X2_train, y_train, cr = (0.2,0.3)), batch_size = 16)
#train_gen = crop_gen_flex_square(X2_train, y_train, cr = 0.2, flip = True, batch_size = 32)
#train_gen = crop_gen_flex(X2_train, y_train, cr = (0.1,0.3), batch_size = 4)

In [206]:
input_shape = (None, None, 512)
#model = fc_bin_m2(Input(shape=input_shape), crop_size)
m_input = Input(shape=input_shape)
model = Model(m_input, fc_bin_m1(m_input))

In [207]:
model.compile(loss='binary_crossentropy',
              optimizer= Adam(lr=0.001),
              metrics=['accuracy'])

In [208]:
model.optimizer.lr = 1e-4
history = model.fit_generator(train_gen,
                    samples_per_epoch=len(X2_train), nb_epoch = 20, verbose = 2,
                    validation_data=(X2_val, y_val))


Epoch 1/20
1s - loss: 0.5402 - acc: 0.7250 - val_loss: 1.2674 - val_acc: 0.5460
Epoch 2/20
0s - loss: 0.2332 - acc: 0.9250 - val_loss: 0.9670 - val_acc: 0.6540
Epoch 3/20
0s - loss: 0.1521 - acc: 0.9500 - val_loss: 0.6967 - val_acc: 0.7120
Epoch 4/20
0s - loss: 0.1037 - acc: 1.0000 - val_loss: 0.5357 - val_acc: 0.7680
Epoch 5/20
0s - loss: 0.0768 - acc: 0.9875 - val_loss: 0.4241 - val_acc: 0.8180
Epoch 6/20
0s - loss: 0.1243 - acc: 0.9625 - val_loss: 0.3722 - val_acc: 0.8380
Epoch 7/20
0s - loss: 0.0387 - acc: 1.0000 - val_loss: 0.3583 - val_acc: 0.8380
Epoch 8/20
0s - loss: 0.1456 - acc: 0.9375 - val_loss: 0.3591 - val_acc: 0.8380
Epoch 9/20
0s - loss: 0.0384 - acc: 0.9875 - val_loss: 0.3298 - val_acc: 0.8440
Epoch 10/20
0s - loss: 0.0408 - acc: 1.0000 - val_loss: 0.2785 - val_acc: 0.8720
Epoch 11/20
0s - loss: 0.0247 - acc: 1.0000 - val_loss: 0.2613 - val_acc: 0.8760
Epoch 12/20
0s - loss: 0.0469 - acc: 0.9875 - val_loss: 0.2681 - val_acc: 0.8700
Epoch 13/20
0s - loss: 0.0244 - acc: 

In [209]:
model.optimizer.lr = 1e-5
history = model.fit_generator(train_gen,
                    samples_per_epoch=len(X2_train), nb_epoch = 10, verbose = 2,
                    validation_data=(X2_val, y_val))

Epoch 1/10
0s - loss: 0.0291 - acc: 1.0000 - val_loss: 0.1747 - val_acc: 0.9320
Epoch 2/10
0s - loss: 0.0183 - acc: 1.0000 - val_loss: 0.1709 - val_acc: 0.9360
Epoch 3/10
0s - loss: 0.0300 - acc: 1.0000 - val_loss: 0.1707 - val_acc: 0.9380
Epoch 4/10
0s - loss: 0.0167 - acc: 1.0000 - val_loss: 0.1689 - val_acc: 0.9360
Epoch 5/10
0s - loss: 0.0310 - acc: 1.0000 - val_loss: 0.1672 - val_acc: 0.9380
Epoch 6/10
0s - loss: 0.0230 - acc: 1.0000 - val_loss: 0.1669 - val_acc: 0.9420
Epoch 7/10
0s - loss: 0.0079 - acc: 1.0000 - val_loss: 0.1689 - val_acc: 0.9340
Epoch 8/10
0s - loss: 0.0062 - acc: 1.0000 - val_loss: 0.1677 - val_acc: 0.9320
Epoch 9/10
0s - loss: 0.0345 - acc: 0.9875 - val_loss: 0.1603 - val_acc: 0.9360
Epoch 10/10
0s - loss: 0.0044 - acc: 1.0000 - val_loss: 0.1562 - val_acc: 0.9400


In [215]:
model.optimizer.lr = 1e-6
history = model.fit_generator(train_gen,
                    samples_per_epoch=len(X2_train), nb_epoch = 5, verbose = 2,
                    validation_data=(X2_val, y_val))

Epoch 1/5
0s - loss: 0.0192 - acc: 1.0000 - val_loss: 0.1494 - val_acc: 0.9580
Epoch 2/5
0s - loss: 0.0066 - acc: 1.0000 - val_loss: 0.1473 - val_acc: 0.9600
Epoch 3/5
0s - loss: 0.0081 - acc: 1.0000 - val_loss: 0.1456 - val_acc: 0.9580
Epoch 4/5
0s - loss: 0.0042 - acc: 1.0000 - val_loss: 0.1448 - val_acc: 0.9560
Epoch 5/5
0s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.1448 - val_acc: 0.9560


In [216]:
pred = model.predict(X2_test)
pred.shape
pred = reduce_conf(pred)
metrics.log_loss(y_test, pred)
pred_class = (pred > 0.5).astype('float32')
metrics.accuracy_score(y_test, pred_class)

(2500, 1)

0.17634539513401687

0.93920000000000003

In [209]:
X2_train.shape
X2_val.shape

(616, 15, 11, 512)

(1500, 15, 11, 512)

In [175]:
#aug_fun(X[i:i+1,::], cr = (0.3,0.1))
pred_lst = [model.predict(aug_fun(X2_test, cr = (0.1,0.3))) for i in range(10)]

In [176]:
sc = [metrics.log_loss(y_test, reduce_conf(pr)) for pr in pred_lst] 
sc
pred = reduce_conf(np.mean(pred_lst, axis = 0))
metrics.log_loss(y_test, pred)
pred_class = (pred > 0.5).astype('float32')
metrics.accuracy_score(y_test, pred_class)

[0.20395648477673531,
 0.20723488982729613,
 0.20619762375652789,
 0.20370713316090405,
 0.20941270342171192,
 0.21822943373098969,
 0.21859982696808875,
 0.21053252746798098,
 0.22544415408000351,
 0.2001515374712646]

0.20312640709616245

0.93079999999999996

In [169]:
X2_val[0,::].shape

int(11 - 0.1*11)

(15, 11, 512)

9

In [667]:
for i in range(10):
    aug_fun(X2_val[:1,::], cr = (0.1,0.3)).shape
#aug_fun((15, 11))

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

(1, 9, 10, 512)

## Submission

We see that feature maps augmentation improves the score on a par of image augmentation but works on an order of magnitude faster.  
Let's of the ultimate test an submit predictions of the baseline model and model with feature maps augmentation.

In [670]:
train_idx, val_idx = next(ShuffleSplit(test_size = 0.05, random_state= 517).split(img_df, img_df.dog))
train_idx.shape
val_idx.shape


(23750,)

(1250,)

In [671]:
def img_arr_gen(df, target_size):
    for r in df.itertuples():
        yield preprocess_image(r.path, target_size = target_size)


In [672]:
df = test_df.iloc[:2]
gen = img_arr_gen(df, target_size = target_size)

In [673]:
tmp = next(gen)
tmp.shape


(1, 374, 500, 3)

In [717]:
df = img_df.iloc[val_idx]
#df = img_df.iloc[:4]
df.shape
gen = img_arr_gen(df, target_size = target_size)
%time X2_val = conv_out.predict_generator(gen, val_samples = df.shape[0], max_q_size = 1)
X2_val.shape
y_val = np.array(df.dog).astype(np.float32)
y_val.shape
np.mean(y_val)

(1250, 2)

CPU times: user 45.6 s, sys: 5.98 s, total: 51.6 s
Wall time: 1min 13s


(1250, 11, 15, 512)

(1250,)

0.5072

In [675]:
df = img_df.iloc[train_idx]
df.shape
gen = img_arr_gen(df, target_size = target_size)
%time X2_train = conv_out.predict_generator(gen, val_samples = df.shape[0], max_q_size = 1)
X2_train.shape
y_train = np.array(df.dog).astype(np.float32)
y_train.shape
np.mean(y_train)


(23750, 2)

CPU times: user 14min 21s, sys: 1min 53s, total: 16min 14s
Wall time: 23min 10s


(23750, 11, 15, 512)

(23750,)

0.49962106

In [676]:
df = test_img_df
df.shape
gen = img_arr_gen(df, target_size = target_size)
%time X2_test = conv_out.predict_generator(gen, val_samples = df.shape[0], max_q_size = 1)
X2_test.shape


(12500, 1)

CPU times: user 7min 32s, sys: 59.3 s, total: 8min 31s
Wall time: 12min 11s


(12500, 11, 15, 512)

### Baseline model

In [682]:
input_shape = (None, None, 512)
#model = fc_bin_m2(Input(shape=input_shape), crop_size)
m_input = Input(shape=input_shape)
model = Model(m_input, fc_bin_m1(m_input))

In [683]:
#model.summary()

In [684]:
model.compile(loss='binary_crossentropy',
              optimizer= Adam(lr=0.001),
              metrics=['accuracy'])

In [685]:
model.optimizer.lr = 1e-4
history = model.fit(X2_train, y_train, validation_data=(X2_val, y_val),
                    nb_epoch = 2, verbose = 2)


Train on 23750 samples, validate on 1250 samples
Epoch 1/2
19s - loss: 0.1089 - acc: 0.9643 - val_loss: 0.0540 - val_acc: 0.9888
Epoch 2/2
16s - loss: 0.0729 - acc: 0.9782 - val_loss: 0.0508 - val_acc: 0.9888


In [686]:
model.optimizer.lr = 1e-5
history = model.fit(X2_train, y_train, validation_data=(X2_val, y_val),
                    nb_epoch = 5, verbose = 2)

Train on 23750 samples, validate on 1250 samples
Epoch 1/5
16s - loss: 0.0609 - acc: 0.9824 - val_loss: 0.0670 - val_acc: 0.9808
Epoch 2/5
16s - loss: 0.0552 - acc: 0.9846 - val_loss: 0.0544 - val_acc: 0.9872
Epoch 3/5
16s - loss: 0.0503 - acc: 0.9855 - val_loss: 0.0503 - val_acc: 0.9856
Epoch 4/5
16s - loss: 0.0476 - acc: 0.9874 - val_loss: 0.0491 - val_acc: 0.9912
Epoch 5/5
16s - loss: 0.0413 - acc: 0.9905 - val_loss: 0.0525 - val_acc: 0.9912


### Augmented feature maps

In [689]:
X2_train.shape

(23750, 11, 15, 512)

In [704]:
train_gen = gen_batch_wrap(conv_crop_gen(X2_train, y_train), batch_size = 64)

In [705]:
input_shape = (None, None, 512)
#model = fc_bin_m2(Input(shape=input_shape), crop_size)
m_input = Input(shape=input_shape)
model = Model(m_input, fc_bin_m1(m_input))

In [706]:
model.compile(loss='binary_crossentropy',
              optimizer= Adam(lr=0.001),
              metrics=['accuracy'])

In [707]:
model.optimizer.lr = 1e-4
history = model.fit_generator(train_gen,
                    samples_per_epoch=len(X2_train), nb_epoch = 5, verbose = 2,
                    validation_data=(X2_val, y_val))


Epoch 1/5


/home/anmiko/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


9s - loss: 0.1158 - acc: 0.9544 - val_loss: 0.0608 - val_acc: 0.9864
Epoch 2/5
7s - loss: 0.0785 - acc: 0.9709 - val_loss: 0.0618 - val_acc: 0.9888
Epoch 3/5
7s - loss: 0.0697 - acc: 0.9744 - val_loss: 0.0545 - val_acc: 0.9856
Epoch 4/5
7s - loss: 0.0639 - acc: 0.9773 - val_loss: 0.0568 - val_acc: 0.9832
Epoch 5/5
6s - loss: 0.0590 - acc: 0.9782 - val_loss: 0.0531 - val_acc: 0.9856


In [708]:
model.optimizer.lr = 1e-5
history = model.fit_generator(train_gen,
                    samples_per_epoch=len(X2_train), nb_epoch = 5, verbose = 2,
                    validation_data=(X2_val, y_val))

Epoch 1/5


/home/anmiko/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


7s - loss: 0.0570 - acc: 0.9801 - val_loss: 0.0512 - val_acc: 0.9856
Epoch 2/5
7s - loss: 0.0510 - acc: 0.9816 - val_loss: 0.0526 - val_acc: 0.9856
Epoch 3/5
7s - loss: 0.0472 - acc: 0.9834 - val_loss: 0.0532 - val_acc: 0.9864
Epoch 4/5
7s - loss: 0.0435 - acc: 0.9848 - val_loss: 0.0509 - val_acc: 0.9872
Epoch 5/5
7s - loss: 0.0412 - acc: 0.9866 - val_loss: 0.0509 - val_acc: 0.9856


In [326]:
#aug_fun(X[i:i+1,::], cr = (0.3,0.1))
pred_lst = [model.predict(aug_fun(X2_test, cr = (0.1,0.3))) for i in range(10)]

In [711]:
pred_lst = [model.predict(aug_fun(X2_val, cr = (0.1,0.3))) for i in range(10)]

In [712]:
sc = [metrics.log_loss(y_val, reduce_conf(pr)) for pr in pred_lst] 
sc
pred = reduce_conf(np.mean(pred_lst, axis = 0))
metrics.log_loss(y_val, pred)
pred_class = (pred > 0.5).astype('float32')
metrics.accuracy_score(y_val, pred_class)

[0.054366152898222206,
 0.054366152898222206,
 0.073901257865875966,
 0.058081215167045591,
 0.050385946031659842,
 0.055275404179841281,
 0.051058247101306915,
 0.050474743369966747,
 0.073901257865875966,
 0.049508453486859796]

0.050945723516494033

0.98640000000000005

In [713]:
pred_lst = [model.predict(aug_fun(X2_test, cr = (0.1,0.3))) for i in range(10)]

In [714]:
pred = reduce_conf(np.mean(pred_lst, axis = 0))

In [709]:
pred = model.predict(X2_test)
pred.shape
stats.describe(pred)
pred = reduce_conf(pred)
stats.describe(pred)

(12500, 1)

DescribeResult(nobs=12500, minmax=(array([  5.07297937e-09], dtype=float32), array([ 1.], dtype=float32)), mean=array([ 0.49884605], dtype=float32), variance=array([ 0.22398372], dtype=float32), skewness=array([ 0.00376293], dtype=float32), kurtosis=array([-1.93805027], dtype=float32))

DescribeResult(nobs=12500, minmax=(array([ 0.01], dtype=float32), array([ 0.99000001], dtype=float32)), mean=array([ 0.49890062], dtype=float32), variance=array([ 0.21772641], dtype=float32), skewness=array([ 0.00376109], dtype=float32), kurtosis=array([-1.94074309], dtype=float32))

In [715]:
test_id = test_img_df.path.str.findall(r'(\d+)\.jpg').apply(lambda x: x[0])
pred.shape
subm_df = pd.DataFrame({'id':test_id, 'label':pred[:,0]})
file = full_path('aug_fm_aug_test_fc_bin_m1.csv','subm')#aug_fm_aug_test
subm_df.to_csv(file, index = False) 
subm_df.describe()

(12500, 1)

,label
count,12500.000000
mean,0.499284
std,0.472516
min,0.010000
25%,0.010000
50%,0.470003
75%,0.990000
max,0.990000


In [332]:
X2_val = conv_out.predict_generator(X_val)
X2_val.shape
X2_train = conv_out.predict(X_train)
X2_train.shape

(1500, 15, 11, 512)

(616, 15, 11, 512)

In [335]:
del X_train
del X_val

In [336]:
del X_test

In [669]:
import gc
gc.collect()

665